In [56]:
import pennylane as qml
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier

In [57]:
# Create a toy dataset
X, y = make_moons(n_samples=20000, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Standardize the data
print(y_train.shape)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

(16000,)


In [58]:
print(X_train)

[[ 1.00726474 -1.46813234]
 [-0.68965642  1.33600835]
 [ 0.00975331  1.73491933]
 ...
 [-1.56147295 -0.08107165]
 [ 0.11203311 -0.98636342]
 [ 0.82547901 -1.40691605]]


In [59]:
n_qubits = 2  # Number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(x, weights):
    # Angle embedding
    qml.templates.AngleEmbedding(x, wires=range(n_qubits))

    # Basic entangler layers
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    
    # Measurement
    return qml.expval(qml.PauliZ(0))


In [60]:
def cost(weights, X, y):
    # Combine X and y into a single array for easier processing
    combined = np.column_stack((X, y))
    
    # Use NumPy's vectorized operations
    predictions = circuit(combined[:, :-1], weights)
    predictions = (predictions + 1) / 2
    
    # Clip predictions to avoid log(0)
    epsilon = 1e-12
    predictions = np.clip(predictions, epsilon, 1 - epsilon)
    
    # Calculate Binary Cross-Entropy loss
    bce_loss = -np.mean(y * np.log(predictions) + (1 - y) * np.log(1 - predictions))
    
    return bce_loss
n_layers = 5
weights = np.random.randn(n_layers, 2, 3)  # (layers, rotations, qubits)

In [61]:
circuit_draw = qml.draw(circuit,level="device")(X_train[0], weights)  # Visualize with the first training sample
print(circuit_draw)


0: ──RX(1.01)───Rot(-0.63,-1.08,-2.38)─╭●─╭X──Rot(0.71,-2.36,0.11)──╭●─╭X──Rot(-0.93,0.64,0.02)─╭●
1: ──RX(-1.47)──Rot(0.99,1.16,0.49)────╰X─╰●──Rot(-0.00,-0.27,0.24)─╰X─╰●──Rot(1.45,0.23,-0.11)─╰X

──╭X──Rot(1.65,1.18,-0.63)─╭●─╭X──Rot(-0.67,-0.22,0.25)─╭●─╭X─┤  <Z>
──╰●──Rot(-1.68,0.32,3.09)─╰X─╰●──Rot(-0.69,-0.27,0.01)─╰X─╰●─┤     


In [62]:
from scipy.optimize import minimize

# Optimize the weights
print(weights.flatten())
result = minimize(lambda w: cost(w.reshape(n_layers, 2, 3), X_train, y_train), 
                  weights.flatten(), method='Powell')
optimized_weights = result.x.reshape(n_layers, 2, 3)



[-0.62897244 -1.07606259 -2.37637009  0.98834329  1.16445786  0.4928361
  0.71038592 -2.35693561  0.10957783 -0.00421221 -0.26755016  0.24010865
 -0.93275001  0.64205606  0.01790422  1.44560115  0.22963679 -0.11200748
  1.64603124  1.17572079 -0.630651   -1.67691345  0.31881486  3.08725156
 -0.67377807 -0.21572929  0.24893356 -0.69146851 -0.27043981  0.00650666]


In [63]:
def predict(X):
    predictions = [circuit(x, optimized_weights) for x in X]
    predictions = np.array(predictions)
    return np.round((predictions + 1) / 2)  # Convert to binary (0 or 1)

# Make predictions
y_pred = predict(X_test)

In [64]:
accuracy_score(y_pred,y_test)

0.8645

In [65]:
# Step 4: Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 5: Choose and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Step 6: Make predictions
y_pred = model.predict(X_test)

# Step 7: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)


Accuracy: 0.89
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.89      0.89      2055
           1       0.88      0.89      0.88      1945

    accuracy                           0.89      4000
   macro avg       0.89      0.89      0.89      4000
weighted avg       0.89      0.89      0.89      4000



In [66]:
# Train the model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Make predictions and evaluate
y_pred_dt = dt_model.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))

Decision Tree Accuracy: 0.998
